# Low Density Parity Check decoding with Belief Propagation

In [21]:
using IndexedFactorGraphs
using BeliefPropagation, BeliefPropagation.Models
using LDPCStorage, AbstractAlgebra
using Random

Load a parity check matrix with $n$ variables and $m$ checks

In [22]:
f = download("https://www.inference.org.uk/mackay/codes/EN/C/816.1A4.843",
    "/tmp/ldpc.alist")
H = load_alist("/tmp/ldpc.alist")
m, n = size(H)

(544, 816)

Compute the corresponding generator matrix

In [36]:
kerH = nullspace(matrix(GF(2), Matrix(H)))[2]
G = collect(kerH .== 1)
@assert size(G, 1) == n
k = size(G, 2)

273

Sample a random codeword $x$ from the ensemble

In [37]:
x = G * bitrand(k) .% 2
@assert sum(H*x .% 2) == 0

Corrupt the codeword by flipping bits with probability $p$, obtain $y$

In [38]:
p = 0.05
noise = rand(n) .< p
y = xor.(x, noise)
println("# of flipped bits: ", sum(noise), "/$n")

# of flipped bits: 37/816


The problem of decoding linear codes is equivalent to that of finding the minimum energy configuration of an Ising model with many-body interactions (read more about this [here](https://arxiv.org/abs/cond-mat/9811406)). Set up the BP computation:

In [39]:
g = FactorGraph(H)
ψ = fill(IsingCoupling(n), m)
ϕ = IsingField.(float.(1 .- 2y))
bp = fast_ising_bp(g, ψ, ϕ);

Run the max-sum algorithm

In [40]:
iters = iterate_ms!(bp; maxiter=100, tol=1e-8)

12

Use the marginals to perform the decoding and get the estimate $\hat{x}$

In [41]:
xhat = [argmax(bᵢ)==2 for bᵢ in beliefs_ms(bp)];

Evaluate the performance by computing the normalized hamming distance $d=\frac{1}{n}\sum_{i=1}^n\delta_{\hat{x}_i,x_i}$

In [42]:
d = sum(xhat .!= x) / n
println("Distortion: ", d)

Distortion: 0.0
